In [3]:
from transformers import AutoImageProcessor, Swinv2ForImageClassification
import torch
from datasets import load_dataset

2024-04-25 12:35:49.638103: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-25 12:35:49.672847: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 12:35:49.672911: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 12:35:49.672935: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 12:35:49.680669: I tensorflow/core/platform/cpu_feature_g

In [4]:
dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
image_processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")
model = Swinv2ForImageClassification.from_pretrained("microsoft/swinv2-tiny-patch4-window8-256")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [9]:
model.swinv2
print(type(model.swinv2))

<class 'transformers.models.swinv2.modeling_swinv2.Swinv2Model'>


In [11]:
from torchvision.models import resnet50, ResNet50_Weights

In [15]:
import torch
from torch import nn
import pytorch_lightning as pl
from transformers import Swinv2Model

class LightningSwinV2(pl.LightningModule):
    def __init__(self, num_labels, learning_rate=2e-5):
        super().__init__()
        self.swinv2 = Swinv2Model.from_pretrained("facebook/swinv2-base-patch4-window8-256")
        self.classifier = nn.Linear(self.swinv2.config.hidden_size, num_labels)
        self.loss = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate

    def forward(self, input_ids, attention_mask=None):
        outputs = self.swinv2(input_ids, attention_mask=attention_mask)
        return self.classifier(outputs.last_hidden_state[:, 0])

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        logits = self(input_ids, attention_mask)
        loss = self.loss(logits, labels)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        logits = self(input_ids, attention_mask)
        loss = self.loss(logits, labels)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

